In [1]:
import pandas as pd
import numpy as np

In [2]:
weather = pd.read_csv("weather.csv")
power = pd.read_csv("cleaned_power.csv")

C:\Users\Wade\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (41,42,49,51,89) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### Convert all dates to datetime object

In [3]:
power["DATE"] = power.apply(lambda x: pd.Timestamp(x["Timestamp"]), axis=1)
weather["DATE"] = weather.apply(lambda x: pd.Timestamp(x["DATE"]), axis=1)

### Ensure total power is included, reduce to match weather interval

In [4]:
#Trim power to match date range
date_mismatch = False
if date_mismatch:
    cutoff = power.index[power['DATE'] == pd.Timestamp('3/1/2020 0:00')].tolist()
    power = power.iloc[:cutoff+1,:]

In [5]:
# Add total power column
if "Total Power" not in power.columns:
    power["Total Power"] = power.apply(lambda x: sum(x[1:10]), axis=1)

In [6]:
power

,Timestamp,Chiller-2 Power,Chiller-3 Power,Chiller-4 Power,Chiller-5 Power,Chiller-6 Power,Chiller-7 Power,Chiller-8 Power,Chiller-9 Power,Chiller-10 Power,Free Cooling HX Power,DATE,Total Power
0,3/1/2019 0:00,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,2019-03-01 00:00:00,0.000000
1,3/1/2019 0:01,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,2019-03-01 00:01:00,0.000000
2,3/1/2019 0:02,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,2019-03-01 00:02:00,0.000000
3,3/1/2019 0:03,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,2019-03-01 00:03:00,0.000000
4,3/1/2019 0:04,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,2019-03-01 00:04:00,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1052637,2/28/2021 23:56,0.0,1367.581299,0.0,0.0,0.0,0.0,0.0,0.0,0.0,486.628296,2021-02-28 23:56:00,1367.581299
1052638,2/28/2021 23:57,0.0,1352.855835,0.0,0.0,0.0,0.0,0.0,0.0,0.0,501.884766,2021-02-28 23:57:00,1352.855835
1052639,2/28/2021 23:58,0.0,1352.390137,0.0,0.0,0.0,0.0,0.0,0.0,0.0,492.000519,2021-02-28 23:58:00,1352.390137
1052640,2/28/2021 23:59,0.0,1386.171753,0.0,0.0,0.0,0.0,0.0,0.0,0.0,500.798767,2021-02-28 23:59:00,1386.171753


### Remove extraneous columns

Only these kept columns do refer to hourly information

In [7]:
weather = weather[["DATE","HourlyAltimeterSetting","HourlyDewPointTemperature","HourlyDryBulbTemperature","HourlyPrecipitation","HourlyPresentWeatherType","HourlyPressureChange","HourlyPressureTendency","HourlyRelativeHumidity","HourlySeaLevelPressure","HourlySkyConditions","HourlyStationPressure","HourlyVisibility","HourlyWetBulbTemperature","HourlyWindDirection","HourlyWindGustSpeed","HourlyWindSpeed"]]

# Clean weather data

**Considerations:**

NaN/blank cannot be assumed to be zero.

s = suspect value (appears with value)

T = trace precipitation amount or snow depth (an amount too small to measure, usually < 0.005 inches water equivalent) (appears instead of numeric value)

M = missing value (appears instead of value)

Blank = value is unreported (appears instead of value)

In [8]:
# There is a lot of NaN values in the weather data. They cannot be assumed to be zero
print(f"Size of weather: {weather.shape[0]}")
print("Number of NaN/blank values in each column:")
weather.isna().sum()

Size of weather: 27186
Number of NaN/blank values in each column:


DATE                             0
HourlyAltimeterSetting        4696
HourlyDewPointTemperature     1815
HourlyDryBulbTemperature      1815
HourlyPrecipitation           6973
HourlyPresentWeatherType     20107
HourlyPressureChange         18567
HourlyPressureTendency       18567
HourlyRelativeHumidity        1815
HourlySeaLevelPressure        6766
HourlySkyConditions           2912
HourlyStationPressure         2079
HourlyVisibility              1810
HourlyWetBulbTemperature      2079
HourlyWindDirection           1850
HourlyWindGustSpeed          23816
HourlyWindSpeed               1817
dtype: int64

### Remove redundant columns

HourlyAltimeterSetting: collinear with HourlyStationPressure

HourlyPressureChange, HourlyPressureTendency: missing many values, redundant with HourlyStationPressure

HourlySeaLevelPressure: a mapping of HourlyStationPressure.

HourlyWindGustSpeed: missing many values, redundant with HourlyWindSpeed

In [9]:
weather_clean = weather.drop(["HourlyAltimeterSetting", "HourlyPressureChange", "HourlyPressureTendency", "HourlySeaLevelPressure", "HourlyWindGustSpeed"],axis=1)

### Remove invalid rows

HourlyDryBulb is one of a set of 4 predictors that consistently occur together in most cases. Rows where they are missing are going to be missing crucial data, and should be removed to be interpolated later.

In [10]:
weather_clean = weather_clean[weather_clean["HourlyDryBulbTemperature"].isna() == False]

In [11]:
print(f"Size of weather: {weather_clean.shape[0]}")
print("Number of NaN/blank values in each column:")
weather_clean.isna().sum()

Size of weather: 25371
Number of NaN/blank values in each column:


DATE                             0
HourlyDewPointTemperature        0
HourlyDryBulbTemperature         0
HourlyPrecipitation           5161
HourlyPresentWeatherType     18292
HourlyRelativeHumidity           0
HourlySkyConditions           1097
HourlyStationPressure          264
HourlyVisibility                 0
HourlyWetBulbTemperature       264
HourlyWindDirection             40
HourlyWindSpeed                  7
dtype: int64

**Notes on remaining features**

*HourlySkyConditions*: It is likely not worth including feature in models, due to difficulty in representation. It appears to have correlation with precipitation and weather metrics, indicating it might be useful in classifying entries for interpollating missing data points.

The documentation suggests that the overall character of the sky can be classified by the topmost/final listed layer. This is how I will represent the feature.

*HourlyPresentWeatherType*: Will need to be expanded as a dummy variable, because one measurement can have multiple values. We should be sure to consider additive effects of combined weather patterns.

Each weather item is indicated to be light, moderate, and heavy. For now, I will avoid including this information. It may be worth expanding this info to additonal dummy variables if some weather type is shown to have usefulness.

*HourlyPrecipitation*: Need to convert T (trace amount) to a number. I will use 0.01, as that seems to be used interchangably in this data set.

### Reduce HourlySkyConditions
Coverage: CLR (clear sky), FEW (few clouds), SCT (scattered clouds), BKN (broken clouds), OVC (overcast), VV (obscured sky)

In [12]:
def reduceSkyConditions(conditions):
    condition = conditions.split(" ")
    if len(condition) >= 2:
        condition = condition[-2]  # Take last condition ([-1] would be height of last condition)
        condition = condition.split(":")[0]
        return str(condition)
    elif len(condition) == 1:
        if condition[0] == "CLR:00":
            return "CLR"
        else:
            return np.NaN
    else:
        return np.NaN

weather_clean["HourlySkyConditions"] = weather_clean.apply(lambda x: reduceSkyConditions(str(x["HourlySkyConditions"])), axis=1)
weather_clean["HourlySkyConditions"].unique()

array(['OVC', nan, 'SCT', 'FEW', 'BKN', 'CLR', 'VV', 'X'], dtype=object)

### Reduce present weather type

Three values given, seperated by pipes. The first (AU, automatic sensor) seems to be most valid for this data set.

In [13]:
# Identify all used AU Codes

au_codes = []

def findAUCodes(conditions):
    condition = conditions.split("|")
    if len(condition) > 1:
        condition = condition[0]
        condition = condition.split(" ")
        types = []
        for cond in condition:
            temp = cond.split(":")[0]
#            if temp[0] == '-' or temp[0] == '+':
#                temp = temp[1:]
            types.append(temp)
        return types
    else:
        return None

out = weather.apply(lambda x: findAUCodes(str(x["HourlyPresentWeatherType"])), axis=1)

for i in out:
    if i is not None:
        for j in i:
            au_codes.append(j)

set(au_codes)

{'',
 '+PL',
 '+RA',
 '+SN',
 '-DZ',
 '-FZ',
 '-PL',
 '-RA',
 '-SN',
 '-TS',
 'BC',
 'BL',
 'BR',
 'DZ',
 'FG',
 'FZ',
 'GR',
 'GS',
 'HZ',
 'MI',
 'PL',
 'RA',
 'SN',
 'SQ',
 'TS',
 'UP',
 'VCTS'}

In [14]:
#include possible AU codes from above block (option to ignore +/-)
au_codes =  [
                'BC',
                'BL',
                'BR',
                'DZ',
                'FG',
                'FZ',
                'GS',
                'HZ',
                'MI',
                'PL',
                'RA',
                'SN',
                'TS',
                'VCTS'  # VCTS will co-occur with TS by this code, should be okay though. Can be improved with assumption that VCTS and TS are mutually exclusive
            ]

def reduceWeatherTypes(conditions):
    condition = conditions.split("|")
    if len(condition) > 1:
        condition = condition[0]
        au_dummy = []
        for au in au_codes:
            au_dummy.append(int(au in condition))
        return au_dummy
    else:
        return [0] * len(au_codes)
    
weather_type_dummies = weather_clean.apply(lambda x: reduceWeatherTypes(str(x["HourlyPresentWeatherType"])), result_type='expand', axis=1)
weather_type_dummies.columns = au_codes
# weather_clean = weather_clean.drop(["HourlyPresentWeatherType"],axis=1)  # Should be done after NaN's are dealt with
weather_clean = pd.concat([weather_clean, weather_type_dummies], axis=1)

### Replace T with number in HourlyPrecipitation

In [15]:
weather_clean.loc[weather_clean.HourlyPrecipitation == 'T', 'HourlyPrecipitation'] = 0.01

# Create hourly summary data

There's probably a ton of ways to do this. I will leverage here the observation that the measurements at 54 minute marks are the most consistently good. I will then map each of these hourly values to the nearest whole hour.

In [16]:
weather_summ = weather_clean[weather_clean["DATE"].dt.minute == 54]

In [17]:
print(f"Size of weather: {weather_summ.shape[0]}")
print("Number of NaN/blank values in each column:")
weather_summ.isna().sum()

Size of weather: 17526
Number of NaN/blank values in each column:


DATE                             0
HourlyDewPointTemperature        0
HourlyDryBulbTemperature         0
HourlyPrecipitation            184
HourlyPresentWeatherType     14209
HourlyRelativeHumidity           0
HourlySkyConditions            216
HourlyStationPressure          216
HourlyVisibility                 0
HourlyWetBulbTemperature       216
HourlyWindDirection              3
HourlyWindSpeed                  3
BC                               0
BL                               0
BR                               0
DZ                               0
FG                               0
FZ                               0
GS                               0
HZ                               0
MI                               0
PL                               0
RA                               0
SN                               0
TS                               0
VCTS                             0
dtype: int64

##### Assumptions to clean summary data:

Many NaN around 2019-11-22. Maintenance? 

HourlyPrecipitation needs further investigation. Must beware T (Trace) values. Appears to be similar gaps to HourlySkyConditions

HourlySkyConditions have gaps of missing data. Perhaps use a classifier to predict for missing values.

In [18]:
# Adjust each datetime to the nearest hour ***ROUNDING DOWN***
weather_summ["DATE"] = weather_summ["DATE"].apply(lambda x: x.replace(minute=0))
#weather_summ.set_index("DATE")

C:\Users\Wade\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [19]:
all = pd.Series(data=pd.date_range(start=weather_summ["DATE"].min(), end=weather_summ["DATE"].max(), freq='H'))
mask = all.isin(weather_summ["DATE"].values)
print(all[~mask])

733     2019-03-31 13:00:00
4494    2019-09-04 06:00:00
5637    2019-10-21 21:00:00
6145    2019-11-12 01:00:00
6397    2019-11-22 13:00:00
6399    2019-11-22 15:00:00
6668    2019-12-03 20:00:00
6669    2019-12-03 21:00:00
6837    2019-12-10 21:00:00
6838    2019-12-10 22:00:00
6839    2019-12-10 23:00:00
6848    2019-12-11 08:00:00
6854    2019-12-11 14:00:00
6855    2019-12-11 15:00:00
6907    2019-12-13 19:00:00
6908    2019-12-13 20:00:00
6932    2019-12-14 20:00:00
12830   2020-08-16 14:00:00
dtype: datetime64[ns]


We need to fill in the missing data points. This can probably be done by using the averages of the other data points nearby.

# Cleaning power data

It seems that some engines tend to show negative numbers consistently or during startup. This needs to be further investigated.

Otherwise, power data can be converted to hourly information either by an average of all measurements in an hour or by sampling at regular hour intervals. Both methods are valid and produce similar results. The biggest proportional difference occurs in the small (sub 5000 Tonnes) power ranges. Ways to calculate each and compare in Excel are given below.

There might be appliations where a max or min value over an hour will be useful as well.

```=AVERAGE(OFFSET(power!$K$2,(ROW(A1)-1)*60,,60,))```

```=OFFSET(power!$K$2,(ROW(B1)-1)*60,,1,)```

```=(B1-A1)/B1```

In [20]:
# No cleaning done yet
power_clean = power[["DATE","Total Power"]]
#power_clean = power.set_index(["DATE"])

In [21]:
# Check for time series continuity
all = pd.Series(data=pd.date_range(start=power_clean["DATE"].min(), end=power_clean["DATE"].max(), freq='H'))
mask = all.isin(power_clean["DATE"].values)
print(all[~mask])

218    2019-03-10 02:00:00
8954   2020-03-08 02:00:00
dtype: datetime64[ns]


In [61]:
# This value should be zero to be consistent with surrounding data values
missing_val = pd.DataFrame([[pd.Timestamp("2019-03-10 02:00:00"), 0],[pd.Timestamp("2020-03-08 02:00:00"), 0]], columns=['DATE','Total Power'])
power_clean = power_clean.append(missing_val, ignore_index=True)

power_clean = power_clean.sort_values(['DATE'])
power_clean = power_clean.reset_index(drop=True)
power_clean = power_clean[:-1]

### Convert to hourly summary

Min and Max need to be fixed, currently they match all rows with min/max values, should only match those with right time and min/max

In [89]:
power_summ_all = power_clean[power_clean['DATE'].dt.minute == 30][['DATE']]
power_summ_all["DATE"] = power_summ_all["DATE"].apply(lambda x: x.replace(minute=0))

In [90]:
power_summ_max = power_clean.groupby([power_clean["DATE"].dt.year, power_clean["DATE"].dt.dayofyear, power_clean["DATE"].dt.hour], sort=False)['Total Power'].max().tolist()
power_summ_all['Total Power (max)'] = power_summ_max

In [91]:
power_summ_min = power_clean.groupby([power_clean["DATE"].dt.year, power_clean["DATE"].dt.dayofyear, power_clean["DATE"].dt.hour], sort=False)['Total Power'].min().tolist()
power_summ_all['Total Power (min)'] = power_summ_min

In [92]:
power_summ_avg = power_clean.groupby([power_clean["DATE"].dt.year, power_clean["DATE"].dt.dayofyear, power_clean["DATE"].dt.hour], sort=False)['Total Power'].mean().tolist()
power_summ_all['Total Power (avg)'] = power_summ_avg

In [93]:
# Get regular interval hourly (on top of the hour)
power_summ_idx = power_clean['DATE'].dt.minute == 54
power_summ_samp = power_clean[power_summ_idx]['Total Power'].tolist()
power_summ_all['Total Power (samp)'] = power_summ_samp

In [ ]:
## Get max hourly (not functional currently)
#power_summ_idx = power_clean.groupby([power_clean["DATE"].dt.year, power_clean["DATE"].dt.dayofyear, power_clean["DATE"].dt.hour], sort=False)['Total Power'].transform(max) == power_clean['Total Power']
#power_summ_max = power_clean[power_summ_idx]
#power_summ_max["DATE"] = power_summ_max["DATE"].apply(lambda x: x.replace(minute=0))

In [92]:
## Get min hourly (not functional currently)
#power_summ_idx = power_clean.groupby([power_clean["DATE"].dt.year, power_clean["DATE"].dt.dayofyear, power_clean["DATE"].dt.hour], sort=False)['Total Power'].transform(min) == power_clean['Total Power']
#power_summ_min = power_clean[power_summ_idx]
#ower_summ_min["DATE"] = power_summ_min["DATE"].apply(lambda x: x.replace(minute=0))

C:\Users\Wade\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [96]:
# Get average hourly
#avg_power_hourly = power_clean.groupby([power_clean["DATE"].dt.year, power_clean["DATE"].dt.dayofyear, power_clean["DATE"].dt.hour]).mean()

#power_summ_idx = power_clean['DATE'].dt.minute == 0
#power_summ_avg = power_clean[power_summ_idx]

#power_summ_avg['Total Power'] = avg_power_hourly['Total Power']

In [97]:
## Get regular interval hourly (on top of the hour)
#power_summ_idx = power_clean['DATE'].dt.minute == 54
#power_summ_reg = power_clean[power_summ_idx]
#power_summ_reg["DATE"] = power_summ_reg["DATE"].apply(lambda x: x.replace(minute=0))

C:\Users\Wade\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


# Merge data sets

In [94]:
data = pd.merge(power_summ_all, weather_summ, how="left", on='DATE')

# Interpolate missing data
Temporary solution for now, fill in NaN rows with row above

In [95]:
# First, column HourlyPresentWeatherType can be removed, if nothin is present, if should be clear
data = data.drop(['HourlyPresentWeatherType'],axis=1)

In [100]:
print(f"Size of data: {data.shape[0]}")
print("Number of NaN/blank values in each column:")
data.isna().sum()

Size of data: 17544
Number of NaN/blank values in each column:


DATE                         0
Total Power (max)            0
Total Power (min)            0
Total Power (avg)            0
Total Power (samp)           0
HourlyDewPointTemperature    0
HourlyDryBulbTemperature     0
HourlyPrecipitation          0
HourlyRelativeHumidity       0
HourlySkyConditions          0
HourlyStationPressure        0
HourlyVisibility             0
HourlyWetBulbTemperature     0
HourlyWindDirection          0
HourlyWindSpeed              0
BC                           0
BL                           0
BR                           0
DZ                           0
FG                           0
FZ                           0
GS                           0
HZ                           0
MI                           0
PL                           0
RA                           0
SN                           0
TS                           0
VCTS                         0
dtype: int64

In [101]:
print(data[data.HourlyWetBulbTemperature.isna()])

Empty DataFrame
Columns: [DATE, Total Power (max), Total Power (min), Total Power (avg), Total Power (samp), HourlyDewPointTemperature, HourlyDryBulbTemperature, HourlyPrecipitation, HourlyRelativeHumidity, HourlySkyConditions, HourlyStationPressure, HourlyVisibility, HourlyWetBulbTemperature, HourlyWindDirection, HourlyWindSpeed, BC, BL, BR, DZ, FG, FZ, GS, HZ, MI, PL, RA, SN, TS, VCTS]
Index: []

[0 rows x 29 columns]


In [99]:
data = data.fillna(method='ffill')

### Export to csv

In [102]:
data.to_csv("merged_data.csv")